### Setup env
This is notebook specific setup, my module path is different on my vm so this is a workaround

In [17]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

### Load Dataset
You can change the type of motion by changing the filepath
Dataset right now just repeats the same motion 100 times

In [25]:
from diffusion.data_loaders.motion_dataset_v2 import MotionDataset
dataset = MotionDataset("data/motions/humanoid3d_cartwheel.txt")
len(dataset), dataset[0], dataset[0].trajectories.shape

Tmp angle [0.0, 0.0, 0.85536, 0.9966429999999997, -0.0070009999999999795, 0.08157, 0.0005729999999999971, 0.042303731260289315, -0.056088768155961526, -0.01172717680484046, -0.014103614145860938, 0.2358842735659614, 0.37124889801787253, -0.6111023347690597, -0.09268300376873025, -0.09541896434572254, 0.585361, 0.1699928747321186, 0.08652758875118252, 0.354108626550405, 0.160215, -0.2285399691330798, -0.39445967594673703, -0.1178224382194308, -0.369571, 0.20448116583595066, -0.12115992907931128, 0.07892319943485762, 0.3736623102073797, -0.010008232584494297, 0.30603690929303384, -0.364281, -0.13425257761871864, -0.004787718949892447, 0.0010873114649849894] 35


(100,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8554,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0037, -0.0062,  0.8563,  ..., -0.2435, -1.1484, -0.8920],
         [ 0.0064, -0.0122,  0.8575,  ..., -0.0278, -1.2998, -0.9550],
         ...,
         [-0.2466,  2.0540,  0.8465,  ..., -0.8048,  0.5575,  1.2816],
         [-0.2435,  2.0658,  0.8467,  ..., -0.7320,  0.5210,  1.2596],
         [-0.2394,  2.0782,  0.8469,  ..., -0.5900,  0.5328,  1.0961]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.5536e-01,  9.9664e-01, -7.0010e-03,
          8.1570e-02,  5.7300e-04,  4.2304e-02, -5.6089e-02, -1.1727e-02,
         -1.4104e-02,  2.3588e-01,  3.7125e-01, -6.1110e-01, -9.2683e-02,
         -9.5419e-02,  5.8536e-01,  1.6999e-01,  8.6528e-02,  3.5411e-01,
          1.6022e-01, -2.2854e-01, -3.9446e-01, -1.1782e-01, -3.6957e-01,
          2.0448e-01, -1.2116e-01,  7.8923e-02,  3.7366e-01, -1.0008e-02,
          3.0604e-01, -3.6428e-01, -1.3425e-01, -4.7877e-03,  1.0873e-03,
 

### Setup Model
Configure your experiment name and savepaths here, they will all be stored under the logs folder later on

In [46]:
import os
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

exp_name = "test-cartwheel-shuffled-128"
savepath = f'/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{exp_name}'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

if not os.path.exists(savepath):
    os.makedirs(savepath)
    os.makedirs(os.path.join(savepath, 'sampled_motions'))

In [27]:
from diffusion.diffuser.models.temporal_v2 import TemporalUnet 

horizon = dataset[0].trajectories.shape[0]
transition_dim = dataset[0].trajectories.shape[1]

model_config = dConfig(
    TemporalUnet,
    savepath=(savepath, 'model_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    cond_dim=transition_dim,
    device=device,
)
model = model_config()


[utils/config ] Config: <class 'diffusion.diffuser.models.temporal_v2.TemporalUnet'>
    cond_dim: 69
    horizon: 160
    transition_dim: 69

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128/model_config.pkl

[ models/temporal ] Channel dimensions: [(69, 128), (128, 256), (256, 512), (512, 1024)]
[(69, 128), (128, 256), (256, 512), (512, 1024)]


In [28]:
from diffusion.diffuser.models.diffusion_v2 import GaussianDiffusion

# model params, I am only using the very basic ones, some params are for conditioning
n_timesteps = 1000
loss_type = 'l2'
clip_denoised = False
predict_epsilon = False
action_weight = 5
loss_weights = None
loss_discount = 1
pos_dim = 35
vel_dim = 34

diffusion_config = dConfig(
    GaussianDiffusion,
    savepath=(savepath, "diffusion_config.pkl"),
    horizon=horizon,
    # transition_dim=transition_dim,
    observation_dim=pos_dim,
    action_dim=vel_dim,
    n_timesteps=n_timesteps,
    loss_type=loss_type,
    clip_denoised=clip_denoised,
    predict_epsilon=predict_epsilon,
    # loss weighting
    action_weight=action_weight,
    loss_weights=loss_weights,
    loss_discount=loss_discount,
    device=device,
)

diffusion = diffusion_config(model)


[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion_v2.GaussianDiffusion'>
    action_dim: 34
    action_weight: 5
    clip_denoised: False
    horizon: 160
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 1000
    observation_dim: 35
    predict_epsilon: False

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128/diffusion_config.pkl



### Setup Training

In [29]:
learning_rate = 2e-4
gradient_accumulate_every = 2
ema_decay = 0.995
sample_freq = 2000
save_freq = 2000
n_train_steps = 1e5
n_saves = 5
save_parallel = False
bucket = None
n_reference = 8
train_batch_size = 32

trainer_config = dConfig(
    dTrainer,
    savepath=(savepath, 'trainer_config.pkl'),
    train_batch_size=train_batch_size,
    train_lr=learning_rate,
    gradient_accumulate_every=gradient_accumulate_every,
    ema_decay=ema_decay,
    sample_freq=sample_freq,
    save_freq=save_freq,
    label_freq=int(n_train_steps // n_saves),
    save_parallel=save_parallel,
    results_folder=savepath,
    bucket=bucket,
    n_reference=n_reference,
)

trainer = trainer_config(diffusion, dataset, renderer=None)


[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-128/trainer_config.pkl



### Test if model and training loop works

In [23]:
import torch
from diffuser.utils import batchify

tunet = model
tunet.to(device)
test_data = dataset[0]
batch = batchify(test_data)
t = torch.randint(0, 1000, (1,), device=device).long().to(device)
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)
res, res.shape

x.shape torch.Size([1, 160, 35])
x.shape torch.Size([1, 35, 160])
x.shape torch.Size([1, 128, 160])
1
xfinal.shape torch.Size([1, 128, 80])
x.shape torch.Size([1, 256, 80])
2
xfinal.shape torch.Size([1, 256, 40])
x.shape torch.Size([1, 512, 40])
3
xfinal.shape torch.Size([1, 512, 20])
x.shape torch.Size([1, 1024, 20])
4
xfinal.shape torch.Size([1, 1024, 20])
xt1.shape torch.Size([1, 1024, 20])
xt2.shape torch.Size([1, 1024, 20])
xt3.shape torch.Size([1, 1024, 20])
pop.shape torch.Size([1, 1024, 20])
pop.shape torch.Size([1, 512, 40])
pop.shape torch.Size([1, 256, 80])


(tensor([[[-0.0948, -0.0691,  0.0749,  ..., -0.1594, -0.1720,  0.0441],
          [-0.2168,  0.0640,  0.1174,  ..., -0.0011, -0.0825,  0.0775],
          [-0.2648, -0.0748,  0.1949,  ..., -0.3040, -0.1790,  0.1174],
          ...,
          [-0.1973,  0.0216,  0.3098,  ..., -0.1037,  0.0096,  0.2913],
          [-0.2072,  0.1525, -0.1111,  ..., -0.0610,  0.3798,  0.3302],
          [-0.2301,  0.0057,  0.0998,  ..., -0.0250, -0.0661,  0.0190]]],
        device='cuda:0', grad_fn=<PermuteBackward0>),
 torch.Size([1, 160, 35]))

In [8]:
from diffuser.utils import report_parameters, batchify

report_parameters(model)

print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')

[ utils/arrays ] Total parameters: 3.96 M
         downs.3.0.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 327.68 k | Conv1d(512, 128, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

### Train Model
It took me 80s to run 1 epoch and results were pretty good from just 1 epoch

In [9]:
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(n_epochs)
n_epochs = 1

for i in range(n_epochs):
    print(f'Epoch {i} / {n_epochs} | {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

trainer.save(n_epochs)

100
Epoch 0 / 1 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel/state_0.pt
0:   0.3086 | a0_loss:   0.0269 | t:   0.4321
100:   0.0675 | a0_loss:   0.0111 | t:   8.1653
200:   0.0279 | a0_loss:   0.0061 | t:   8.1041
300:   0.0137 | a0_loss:   0.0037 | t:   7.9897
400:   0.0073 | a0_loss:   0.0024 | t:   8.1874
500:   0.0044 | a0_loss:   0.0016 | t:   8.3597
600:   0.0030 | a0_loss:   0.0011 | t:   8.4126
700:   0.0022 | a0_loss:   0.0008 | t:   8.2396
800:   0.0017 | a0_loss:   0.0006 | t:   8.4983
900:   0.0014 | a0_loss:   0.0004 | t:   8.5166
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel/state_1.pt


### (Alternatively) load a checkpoint

In [30]:
# from diffusion.diffuser.utils import load_diffusion
# diffusion_experiment = load_diffusion(
#     "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser", dataset=dataset, epoch=1)

# renderer = diffusion_experiment.renderer
# model = diffusion_experiment.trainer.ema_model

# Optionally load a checkpoint
trainer.load(3)
model = trainer.ema_model

### Sample from model
We want to see what the model does if it starts with a motion, could it convert one motion to another? Or blend them in someway?

Start from existing motion data instead of noise, we want to mix walking and doing a cartwheel

In [41]:
from diffusion.data_loaders.motion_dataset_v2 import MotionDataset

walk_dataset = MotionDataset("data/motions/humanoid3d_walk.txt")
len(walk_dataset), walk_dataset[0], walk_dataset[0].trajectories.shape

Tmp angle [0.0, 0.0, 0.847532, 0.998678, 0.01410399999999995, 0.049422999999999995, -0.0006980000000000042, 0.019374995056800275, 0.008037254877450587, -0.09523902811084285, -0.0, 0.0, -0.0, -0.15553532463598624, 0.23919429256424163, 0.20739656997070288, 0.170571, 0.3529631848273465, -0.2610682953696931, -0.24560532144975333, 0.581348, 0.02035205257945668, -0.5175742452141794, -0.11376339039728192, -0.249116, 0.020556236630260034, -0.019534498786735962, 0.0655269812790598, -0.05606350142619236, 0.15209578259548684, 0.1827420948157945, -0.391532, 0.1931167851688944, -0.2978918547932108, -0.08305715225197069] 35


(100,
 Batch(trajectories=tensor([[ 0.0000,  0.0000,  0.8475,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0415, -0.0047,  0.8466,  ...,  0.4419, -0.9102,  1.1832],
         [ 0.0817, -0.0105,  0.8483,  ...,  1.9768, -2.7716,  1.5138],
         ...,
         [ 0.9535,  0.0164,  0.8741,  ..., -0.0242,  0.5358,  0.5449],
         [ 0.9810,  0.0152,  0.8736,  ...,  0.0819,  0.5424,  0.2741],
         [ 1.0080,  0.0138,  0.8721,  ..., -0.0949,  0.5196,  0.1573]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.4753e-01,  9.9868e-01,  1.4104e-02,
          4.9423e-02, -6.9800e-04,  1.9375e-02,  8.0373e-03, -9.5239e-02,
         -0.0000e+00,  0.0000e+00, -0.0000e+00, -1.5554e-01,  2.3919e-01,
          2.0740e-01,  1.7057e-01,  3.5296e-01, -2.6107e-01, -2.4561e-01,
          5.8135e-01,  2.0352e-02, -5.1757e-01, -1.1376e-01, -2.4912e-01,
          2.0556e-02, -1.9534e-02,  6.5527e-02, -5.6064e-02,  1.5210e-01,
          1.8274e-01, -3.9153e-01,  1.9312e-01, -2.9789e-01, -8.3057e-02,
 

Extend the walk to be the same shape as the cartwheel

In [42]:
walk_traj = walk_dataset[0].trajectories
walk_traj.shape

torch.Size([32, 69])

In [54]:
walk_traj = walk_dataset[0].trajectories
walk_traj = torch.cat([walk_traj] * 5, dim=0)
pos_diff = walk_traj[-1, :3] - walk_traj[0, :3]
walk_traj[32:, :3] += pos_diff[:]
walk_traj[64:, :3] += pos_diff[:]
walk_traj[96:, :3] += pos_diff[:]
walk_traj[128:, :3] += pos_diff[:]
import numpy as np

savepath = "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/0-test"


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = walk_traj
    pos_data = pos_data[:, :35].cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")


save_motions(
    None,
    f"{savepath}",
    filename="test2.npy",
)
walk_traj.unsqueeze_(0)
walk_traj.shape

Motion saved as test2.npy


torch.Size([1, 160, 69])

In [52]:
from diffuser.utils import batchify
test = dataset[0]
batch = batchify(test)
sample = diffusion.p_sample_loop(batch.trajectories.shape, batch.conditions, starting_motion=walk_traj, max_timesteps=2)


                                                                                                    
1 / 2 [##############################                              ]  50% | 13.1 Hz
t : 1 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 2 [############################################################] 100% | 22.0 Hz
t : 0 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
[ Progress ] 2 / 2 | t : 0 | vmax : 0.0 | vmin : 0.0 | 22.0 Hz


### Save results to logs

In [53]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0)[:, :35].cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="starting-walk-motion.npy")

Motion saved as starting-walk-motion.npy
